In [5]:
from PIL import Image, ImageDraw
from matplotlib import pyplot as plt
import numpy as np
from scipy.spatial import cKDTree

In [6]:
width, height = 3200, 1600

points = np.array([
    [1500, 300],
    [2400, 1120],
    [800, 1320],
    [750, 450],
    [400, 120],
    [2200, 600],
    [380, 1000],
    [1750, 800],
    [1250, 1100],
    [1900, 1300],
    [1100, 700],
    [1000, 200],
    [2000, 250],
    [110, 300],
    [2900, 1400],
    [250, 600],
    [150, 1350],
    [2500, 400],
    [2650, 840],
    [3000, 120]
])
cyan = (12, 236, 221)
yellow = (255, 243, 56)
pink = (196, 0, 255)
magenta = (255, 103, 231)

colors = [
    cyan,
    cyan,
    yellow,
    pink,
    magenta,
    magenta,
    cyan,
    yellow,
    pink,
    magenta,
    magenta,
    yellow,
    pink,
    cyan,
    yellow,
    yellow,
    pink,
    cyan,
    pink,
    yellow
]

tree = cKDTree(points)

# colors = np.array(colors)/255

# plt.scatter(*points.T, c=colors)
# plt.show()

In [7]:
def d_matrix(x, y):
    i, j = np.indices((height, width))
    i -= y
    j -= x
    return np.sqrt(i*i+j*j)

dist = np.dstack([d_matrix(*p) for p in points]).T
dist = np.swapaxes(dist, 1, 2)

dist_min = np.minimum.reduce(dist, axis=0)
ind_min = dist == dist_min

im = np.zeros((height, width, 3), dtype=np.int32)
for i in range(len(points)):
    im[dist[i] == dist_min] = colors[i]

dist_min = np.repeat(dist_min[:, :, np.newaxis], 3, axis=-1)

for d_max in range(10, 800, 25):
    temp_im = np.where(dist_min < d_max, im, 255).astype(np.uint8)
    temp_im = Image.fromarray(temp_im)
    draw = ImageDraw.Draw(temp_im)

    for i, point in enumerate(points):
        circle_size = np.array([100, 100])
        p1 = point - circle_size/2
        p2 = point + circle_size/2
        
        draw.ellipse((*p1, *p2), fill=colors[i], outline=(0, 0, 0), width=10)

    temp_im = temp_im.resize((800, 400), resample=Image.ANTIALIAS)
    temp_im.save(f"figure_3/scene1_{d_max}.png")